In [1]:
import sqlite3
import pandas as pd

sqlite3 is a built-in Python library for working with SQLite databases.
SQLite is serverless (unlike MySQL or PostgreSQL), meaning it stores everything in a single file (.db) instead of needing a separate database server. Adds to the convenience of this small scale implementation.

In [3]:
def setup_database():
    conn = sqlite3.connect("sample.db")  # Connects to SQLite database (creates it if not exists).
    cursor = conn.cursor()  # Creates a cursor to execute SQL commands. It basically acts as a tool that sends SQL queries to the database and retrieves results.
    
    # Creating sample tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            department TEXT, 
            salary INTEGER
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY, 
            customer_name TEXT, 
            amount REAL
        )
    """)
    
    # Inserting sample data
    cursor.executemany("INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)", [
        ("Alice", "HR", 50000),
        ("Bob", "Engineering", 75000),
        ("Charlie", "Marketing", 60000)
    ])
    
    cursor.executemany("INSERT INTO orders (customer_name, amount) VALUES (?, ?)", [
        ("Alice", 120.50),
        ("Bob", 300.00),
        ("Charlie", 150.75)
    ])
    
    conn.commit()
    conn.close()

def execute_query(query):
    #This will be our function to execute a query and get the result.
    conn = sqlite3.connect("sample.db")
    result = pd.read_sql_query(query, conn)
    #This line executes an SQL query on the SQLite database and returns the result as a Pandas DataFrame.
    conn.close()
    return result

# We setup the database
setup_database()

# Example test query
query = "SELECT * FROM employees WHERE department = 'Engineering'"
print(execute_query(query))


   id name   department  salary
0   2  Bob  Engineering   75000


## Importing and loading a NL to SQL model 

In [37]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Force CPU (since your laptop doesn't have enough VRAM)
device = torch.device("cpu")

# Model name
model_name = "SwastikM/bart-large-nl2sql"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

#transformers → A library from Hugging Face for using pre-trained AI models.
#AutoModelForSeq2SeqLM → Loads a sequence-to-sequence model (T5 in this case) that translates text to SQL.
#AutoTokenizer → Converts natural language text into tokens that the model understands.

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

## Creating a function to generate the apt query when provided with user prompt and context

In [39]:
# Function to generate SQL
def generate_sql(nl_query, sql_schema):
    query_question_with_context = f"sql_prompt: {nl_query} sql_context: {sql_schema}"
    inputs = tokenizer(query_question_with_context, return_tensors="pt").to(device)
    #Converts text into numerical token IDs (which the model understands).
    #return_tensors="pt" → Returns PyTorch tensors (needed for model input).
    # Generate SQL
    outputs = model.generate(inputs.input_ids, max_new_tokens=100, do_sample=False)
    #The model processes the input and generates output tokens.
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #Converts tokenized output back to human-readable text.
    return sql_query

# Example Usage
nl_query = "Show me all employees with a salary above 50000"
sql_schema = "CREATE TABLE employees (id INT, name TEXT, department TEXT, salary INT);"

sql_output = generate_sql(nl_query, sql_schema)
print("Generated SQL Query:", sql_output)




Generated SQL Query: SELECT * FROM employees WHERE salary > 50000;


## Running the query in connection with our sql database.

In [40]:
def process_nl_query(nl_query):
    sql_schema = "CREATE TABLE employees (id INT, name TEXT, department TEXT, salary INT);"  # Ensure schema is passed
    sql_query = generate_sql(nl_query, sql_schema)
    print("Generated SQL Query:", sql_query)  # Debugging
    try:
        result = execute_query(sql_query)
        return result
    except Exception as e:
        return f"Error executing query: {e}"

nl_query = "List all employees earning more than 50000"
result = process_nl_query(nl_query)
print(result)


Generated SQL Query: SELECT * FROM employees WHERE salary > 50000;
   id     name   department  salary
0   2      Bob  Engineering   75000
1   3  Charlie    Marketing   60000


In [ ]:
#pip install streamlit
#import streamlit as st

#st.title("Natural Language to SQL Converter")

#nl_query = st.text_input("Enter your natural language query:")

#if st.button("Generate SQL and Execute"):
#    if nl_query:
#        sql_query = generate_sql(nl_query)
#        st.write("Generated SQL:", sql_query)
#        try:
#            result = execute_query(sql_query)
#            st.dataframe(result)
#        except Exception as e:
#            st.error(f"Error executing query: {e}")
#streamlit run app.py##
